# Notebook Setup

In [ ]:
import pandas as pd
import seaborn as sb

from IPython.display import display

In [ ]:
MEASURES_FILENAME = "./measures.csv"
CLOCK_ADJUST_RECTYPE = "CLOCK_ADJUSTED"
MEASURE_RECTYPE = "REGULAR"

COSC_LIMITS_SEC_DAY = (-4, 6)

GRAPH_STYLE = "whitegrid"

In [ ]:
sb.set_theme("notebook", GRAPH_STYLE)
sb.set_style()

pd.options.display.max_rows = 6

# Read Data

In [ ]:
# Read the csv
measures = pd.read_csv(MEASURES_FILENAME)

# Put data back in chronological order
measures = measures.iloc[::-1].reset_index(drop=True)

In [ ]:
# Rename columns to make coding comfy
measures = measures.rename(columns={
    "Type of record":"rectype",
    "Offset (seconds)":"offset"
})

In [ ]:
measures = measures.astype({"rectype":"category"})

## Extract machine readable timestamps (utc)

In [ ]:
measures["timestamp"] = pd.to_datetime(measures["Timestamp (Epoch Time)"], unit="ms")

## Create Unique Chain Numbers

In [ ]:
measures["seriesID"] = measures.rectype.eq(CLOCK_ADJUST_RECTYPE).cumsum().shift(1, fill_value=0)

In [ ]:
measures.groupby("seriesID").head(1)

In [ ]:
seriesStarts = measures.groupby("seriesID").timestamp.min()
measures["adjustmentAge"] = measures.timestamp - measures.seriesID.map(seriesStarts)
measures["adjustmentDays"] = measures.adjustmentAge.dt.total_seconds() / 3600 / 24
measures




# Plot Offsets

In [ ]:
f = sb.relplot(data=measures, x="timestamp", y="offset", hue="seriesID", kind="line", aspect=2)
f.figure.autofmt_xdate()

# Plot Superposed Adjustment Cycles

In [ ]:
sb.set_style("ticks")
f = sb.lmplot(data=measures, x="adjustmentDays", y="offset", hue="seriesID")

maxDays = measures.adjustmentDays.max()
minOffset = measures.offset.min()

limitsX = [0, maxDays]
limitYMin = [0, maxDays * COSC_LIMITS_SEC_DAY[0]]
limitYMax = [0, maxDays * COSC_LIMITS_SEC_DAY[1]]

f.ax.fill_between(limitsX, limitYMin, limitYMax, color="green", alpha=0.2, zorder=0)

for i in range(-30,31):
    f.ax.axline((0,0), (1/100, i/100), c="lightgrey", lw=0.5, ls="--")

f.ax.axhline(0, c="black", lw=2, ls="--")

# Get Drift Rates

In [ ]:
from scipy.stats import linregress

def chainStats(df):
    print(f"Doing chain stats on {df.seriesID.unique()[0]}")
    res = linregress(df.adjustmentDays, df.offset)
    print(res)
    return pd.Series({"slope":res.slope, "intercept":res.intercept, "R":res.rvalue})

measures.query("rectype == @MEASURE_RECTYPE").groupby("seriesID").apply(chainStats)